#Q2

Justification for Using 1 Bit:

Minimal Visual Impact:
Using only 1 Least Significant Bit (LSB) per color channel ensures that the change in pixel value is very small (only 1 out of 256 possible values in an 8-bit channel). This change is usually imperceptible to the human eye, thus preserving the original image quality.

Sufficient Capacity for Small Messages:
For short text messages, 1 bit per pixel is generally sufficient. The slight capacity provided by 1 bit is enough to embed the message along with any necessary metadata (like the message length) without overloading the image.

Lower Risk of Detection:
By altering only the least significant bit, the overall appearance of the image remains almost unchanged, reducing the risk that someone will notice any anomalies or suspect that a hidden message exists.

Trade-off Consideration:
While using 2 bits per pixel would increase the capacity for larger messages, it also increases the likelihood of introducing noticeable distortions in the image. For applications where image fidelity is important, especially in something as personal as a selfie, it’s ideal to minimize any changes that could degrade the image quality.

Overall, using 1 bit strikes an optimal balance between hiding enough data for the intended message and maintaining the original image’s visual integrity.

Justification:

1 bit changes each pixel’s color value by at most 1 (in decimal), so the difference is usually invisible. This is good for smaller messages.
2 bits can store double the data but might cause slight color variations in uniform areas.

In [ ]:
###22i-2034 SOHAIB SHAHZAD DS-A

import cv2
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# Global Helper Functions for Text Steganography (Options 1 & 2)
# ============================================================

def text_to_binary(message):
    
    return ''.join([format(ord(c), '08b') for c in message])

def binary_to_text(binary_str):
  
    chars = []
    for i in range(0, len(binary_str), 8):
        byte = binary_str[i:i+8]
        chars.append(chr(int(byte, 2)))
    return ''.join(chars)

def encode_text_in_image(img, secret_text, bits=1):
    
    encoded_img = img.copy()  # Work on a copy
    h, w, c = encoded_img.shape
    total_pixels = h * w * c

    secret_bin = text_to_binary(secret_text)
    msg_len = len(secret_bin)
    msg_len_bin = format(msg_len, '032b')  # 32-bit length header

    data_to_hide = msg_len_bin + secret_bin

    if bits * total_pixels < len(data_to_hide):
        raise ValueError("Not enough capacity in the image to hide the message with {} bits.".format(bits))

    flat_img = encoded_img.reshape(-1)
    for i in range(len(data_to_hide)):
        pixel_val = flat_img[i]
        pixel_bin = format(pixel_val, '08b')
        new_pixel_bin = pixel_bin[:-bits] + data_to_hide[i:i+bits]
        flat_img[i] = int(new_pixel_bin, 2)
    encoded_img = flat_img.reshape(h, w, c)
    return encoded_img

def decode_text_from_image(img, bits=1):
    
    h, w, c = img.shape
    flat_img = img.reshape(-1)

    length_bin = ""
    for i in range(32):
        pixel_val = flat_img[i]
        pixel_bin = format(pixel_val, '08b')
        length_bin += pixel_bin[-bits:]
    msg_length = int(length_bin, 2)

    message_bits = ""
    for i in range(32, 32 + msg_length):
        pixel_val = flat_img[i]
        pixel_bin = format(pixel_val, '08b')
        message_bits += pixel_bin[-bits:]
    hidden_text = binary_to_text(message_bits)
    return hidden_text

# ============================================================
# Option 1: Capture Selfie and Hide/Extract Text Message
# ============================================================

def option1():
    # --- Capture a Selfie ---
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Could not open webcam")
        return
    print("Webcam opened successfully")
    ret, frame = cap.read()
    if ret:
        cv2.imwrite('selfie.jpg', frame)
        print("Selfie captured and saved as 'selfie.jpg'.")
    else:
        print("Failed to capture an image from the webcam.")
        cap.release()
        return
    cap.release()
    cv2.destroyAllWindows()

    # --- Hide a Text Message in the Selfie ---
    original_image = cv2.imread('selfie.jpg')
    if original_image is None:
        print("Error: Could not load 'selfie.jpg'.")
        return
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    # You can change the secret message or even prompt the user
    secret_message = "  secret  code  !!!!"
    stego_image_rgb = encode_text_in_image(original_image_rgb, secret_message, bits=1)
    print("Secret message embedded successfully!")

    # --- Display Original and Stego Images ---
    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.imshow(original_image_rgb)
    plt.title("Original Selfie")
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(stego_image_rgb)
    plt.title("Stego Selfie (Message Hidden)")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    # --- Extract and Display the Hidden Message ---
    recovered_message = decode_text_from_image(stego_image_rgb, bits=1)
    print("Recovered Hidden Message:", recovered_message)

# ============================================================
# Option 2: Hide and Extract Text in Any Image
# ============================================================

def option2():
    image_path = input("Enter the path to your image (e.g., 'image.jpg'): ")
    secret_message = input("Enter the text you want to hide: ")
    bits_to_use = 1
    print(f"Using {bits_to_use} bit(s) for LSB steganography.")

    original_image = cv2.imread(image_path)
    if original_image is None:
        print("Error: Could not load the image. Check the file path.")
        return
    original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

    stego_image_rgb = encode_text_in_image(original_image_rgb, secret_message, bits=bits_to_use)
    print("Message hidden successfully in the image.")

    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.imshow(original_image_rgb)
    plt.title("Original Image")
    plt.axis('off')
    plt.subplot(1,2,2)
    plt.imshow(stego_image_rgb)
    plt.title("Stego Image (Message Hidden)")
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    recovered_message = decode_text_from_image(stego_image_rgb, bits=bits_to_use)
    print("Recovered Hidden Message:", recovered_message)

# ============================================================
# Option 3: Hide and Extract an Image Inside Another Image
# ============================================================

def option3():
    cover_path = input("Enter the cover image file path (e.g., 'cover.webp'): ")
    secret_path = input("Enter the secret image file path (e.g., 'secret.webp'): ")

    cover_img = cv2.imread(cover_path)
    if cover_img is None:
        print("Error: Unable to load cover image.")
        return
    secret_img = cv2.imread(secret_path)
    if secret_img is None:
        print("Error: Unable to load secret image.")
        return

    # Resize secret image if dimensions differ
    if cover_img.shape != secret_img.shape:
        secret_img = cv2.resize(secret_img, (cover_img.shape[1], cover_img.shape[0]))
        print("Secret image has been resized to match the cover image dimensions.")

    # --- Hiding Process ---
    # Clear last 2 bits of cover image
    cover_cleared = cover_img & 0xFC  # 0xFC = 11111100
    # Extract the 2 most significant bits from secret image
    secret_bits = secret_img >> 6
    # Combine to embed secret image into cover image
    hidden_img = cover_cleared | secret_bits

    # Convert for display (BGR -> RGB)
    cover_img_rgb = cv2.cvtColor(cover_img, cv2.COLOR_BGR2RGB)
    secret_img_rgb = cv2.cvtColor(secret_img, cv2.COLOR_BGR2RGB)
    hidden_img_rgb = cv2.cvtColor(hidden_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(18,6))
    plt.subplot(1,3,1)
    plt.imshow(cover_img_rgb)
    plt.title("Cover Image")
    plt.axis("off")
    plt.subplot(1,3,2)
    plt.imshow(secret_img_rgb)
    plt.title("Secret Image")
    plt.axis("off")
    plt.subplot(1,3,3)
    plt.imshow(hidden_img_rgb)
    plt.title("Hidden Image (Secret embedded)")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

    # --- Extraction Process ---
    extracted_secret_bits = hidden_img & 0x03  # 0x03 = 00000011 (last 2 bits)
    # Scale the bits to full intensity range (0,85,170,255)
    extracted_secret = extracted_secret_bits * 85
    extracted_secret_rgb = cv2.cvtColor(extracted_secret, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8,6))
    plt.imshow(extracted_secret_rgb)
    plt.title("Extracted Secret Image")
    plt.axis("off")
    plt.show()

# ============================================================
# Option 4: Superimpose Two Images Using 4 Bits Each
# ============================================================

def option4():
    img1_path = input("Enter the path or name of the first image (e.g., 'image1.webp'): ")
    img2_path = input("Enter the path or name of the second image (e.g., 'image2.webp'): ")

    img1 = cv2.imread(img1_path)
    if img1 is None:
        print("Error: Unable to load the first image.")
        return
    img2 = cv2.imread(img2_path)
    if img2 is None:
        print("Error: Unable to load the second image.")
        return

    img1_rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2_rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.imshow(img1_rgb)
    plt.title("Image 1")
    plt.axis("off")
    plt.subplot(1,2,2)
    plt.imshow(img2_rgb)
    plt.title("Image 2")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

    # Resize img2 if dimensions differ from img1
    if img1.shape != img2.shape:
        img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

    # Clear lower 4 bits of image 1 and extract upper 4 bits of image 2
    img1_upper = img1 & 0xF0       # 0xF0 = 11110000
    img2_upper = (img2 & 0xF0) >> 4  # Shift right by 4 bits
    superimposed = img1_upper | img2_upper

    superimposed_rgb = cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8,6))
    plt.imshow(superimposed_rgb)
    plt.title("Superimposed Image")
    plt.axis("off")
    plt.show()

# ============================================================
# Main Menu-Driven Program
# ============================================================

def main():
    while True:
        print("\nMenu:")
        print("1. Capture Selfie and Hide/Extract Text Message")
        print("2. Hide/Extract Text in any Image")
        print("3. Hide and Extract an Image inside another Image")
        print("4. Superimpose Two Images using 4 Bits each")
        print("5. Exit")
        choice = input("Enter your choice (1-5): ")
        if choice == '1':
            option1()
        elif choice == '2':
            option2()
        elif choice == '3':
            option3()
        elif choice == '4':
            option4()
        elif choice == '5':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()



Menu:
1. Capture Selfie and Hide/Extract Text Message
2. Hide/Extract Text in any Image
3. Hide and Extract an Image inside another Image
4. Superimpose Two Images using 4 Bits each
5. Exit
